*** Export HTML ***
*******************

>  # ****Quelles sont les politiques et comportements qui ont le mieux servi à  combattre la maladie et quels sont celles qu'il faut à tout pris éviter?****

> ****Introduction****

> Le COVID-19 est un virus de la famille des coronavirus. Premièrement apparu à Wuhan, (Chine) en Décembre 2019. Il ne lui aura fallu que quelques mois pour contaminer la planète tout entière et s'inscrire dans l'histoire comme l'une des pandémies les plus mortelles du XXI eme siècle.

<img src="http://www.sciencesetavenir.fr/assets/img/2020/02/25/cover-r4x3w1000-5e5fc2725e88b-f23d9e36f6e0c38d2b1121a43e44b24d6c3f56bf-jpg.jpg" alt="Image d'une représentation du virus" style="width: 300px;"/>


> ****Quels sont les symptômes du coronavirus?****

>  Les symptômes les plus courants de la COVID-19 sont la fièvre, la fatigue et une toux sèche. Certains patients présentent des douleurs, une congestion nasale, un écoulement nasal, des maux de gorge ou une diarrhée. Ces symptômes sont généralement bénins et apparaissent de manière progressive. Certaines personnes, bien qu’infectées, ne présentent aucun symptôme et se sentent bien. Les personnes âgées et celles qui ont d’autres problèmes de santé (hypertension artérielle, problèmes cardiaques ou diabète) ont plus de risques de présenter des symptômes graves.

> ****Comment se propage le coronavirus?****

>  Les études menées à ce jour semblent indiquer que le virus responsable de la COVID-19 est principalement transmissible par contact avec des gouttelettes respiratoires, plutôt que par voie aérienne. Les coronavirus sont une grande famille de virus couramment présents chez les chauves-souris et chez d’autres animaux. Occasionnellement ces virus infectent des êtres humains qui peuvent à leur tour propager l’infection.

> ****Combien de temps dure la période d'incubation de la COVID-19?****

> La période d’incubation est le temps qui s’écoule entre l’infection et l’apparition des symptômes de la maladie. On estime actuellement que la période d’incubation de la COIVD-19 dure de 1 à 14 jours et le plus souvent autour de cinq jours. Ces estimations seront actualisées à mesure que de nouvelles données seront disponibles.


>  Ces informations ont été tiré du site officiel de l'OMS

Source:
https://www.who.int/fr/emergencies/diseases/novel-coronavirus-2019/advice-for-public/q-a-coronaviruses


> ****Comment va t-on procéder.****

> Tout d'abord on va réunir les données des pays, présenter notre jeux de données. 
On veux mesurer plusieurs choses:
> * Les facteurs de ****propagation du virus****, on va utiliser des variables, comme le taux brut de confirmés
> * Les facteurs de ****mauvaise gérence des patients**** avec des ****taux de guérisson/létalité/mortalité****

> Ensuite dans un premier temps nous allons analyser les pays qui ont le moins bien gérés l'épidémie un par un, et nous allons relever ce qu'ils ont en commun
Puis de même, nous analyserons les pays qui ont le mieux gérés un par un ,pour relever ce qu'ils ont en commun


In [ ]:
import numpy as np 
import pandas as pd 
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
# https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html (pour les plots)
import math
import matplotlib.dates as mdates
import PIL
import io

from IPython.display import Markdown
def bold(string):
    display(Markdown(string)) # Pour les caractères gras

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
covid19_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

covid19_data.rename(columns={"Country/Region":"Country_Region"}, inplace=True)
covid19_data.replace("Congo (Kinshasa)","DR Congo",inplace=True)
covid19_data.replace("Congo (Brazzaville)","Congo",inplace=True)
covid19_data.replace("Curacao","Curaçao",inplace=True)
covid19_data.replace("Faroe Islands","Faeroe Islands",inplace=True)
covid19_data.replace("Ivory Coast","Côte d'Ivoire", inplace=True)
covid19_data.replace("Macau","Macao",inplace=True)
covid19_data.replace("US","United States",inplace=True)
covid19_data.replace("UK","United Kingdom",inplace=True)
covid19_data.replace("Mainland China","China",inplace=True)
covid19_data.replace("St. Martin","Saint Martin",inplace=True)
covid19_data.replace("Saint Vincent and the Grenadines","St. Vincent & Grenadines",inplace=True)
covid19_data.replace("East Timor","Timor-Leste",inplace=True)
covid19_data.replace("Sao Tome and Principe","Sao Tome & Principe",inplace=True)
covid19_data.replace("Saint Kitts and Nevis","Saint Kitts & Nevis",inplace=True)


covid19_data.head()

> Nous avons un numéro de serie,(inutile), la province du pays, le pays, la date ainsi que le nombre de confirmés, de morts, et de guérris à cette date.
Petite remarque, ils ont compté le nombre de confirmés différement pour la chine et les états unis. Ils ont regroupés les cas par provinces et non pas à l'échelle du pays, il faudra donc gérer ça lors d'u néttoyage des données. Dans notre dataset, il y a des pays qui ont plusieurs noms par exemple pour les Bahamas, on a Bahamas, The Bahamas, Bahamas,The, il faudra aussi s'occuper de ça

In [ ]:
china_coro = covid19_data[covid19_data["Country_Region"]== "China"].copy()
china_coro.head() # Je prends les données de la chine seulement

In [ ]:
us_coro = covid19_data[covid19_data["Country_Region"]== "United States"].copy()
us_coro.head()   # Je prends les données de US seulement

In [ ]:

us_china_data = pd.concat([us_coro,china_coro],join='inner') #Je les concatene 
us_china_data = us_china_data.groupby(["Province/State","Country_Region"])["Last Update","Confirmed","Deaths","Recovered"].max()
us_china_data = us_china_data.groupby(["Country_Region"])["Confirmed","Deaths","Recovered"].sum().reset_index(drop=False)
us_china_data.set_index(["Country_Region"],drop=False,inplace=True)
us_china_data.head()
# Je les trie avec groupby, je prends le maximum, ensuite j'additione les valeurs des provinces entre elles
# Comme ça j'ai l'information au niveau du pays

In [ ]:
# Pour les autres pays le décompte a déjà été fait au niveau du pays, je n'ai qu'a prendre 
# la date la plus récente, et pour être sur, je prends aussi la valeur maximal de 
# Confirmed, Deaths et Recovered

with_provinces = covid19_data.groupby(["Country_Region"])["Last Update","Confirmed","Deaths","Recovered"].max()
print(with_provinces.shape)

In [ ]:
with_provinces.drop(["United States","China","Diamond Princess","MS Zaandam","Others","('St. Martin',)"],inplace=True) 
#Je retire  les deux premiers ils ont déjà été traité, et j'avais remarqué d'autres valeurs qui n'étaient pas
#des pays, MS Zaandam et Diamond Princess sont des Bateaux de croisière ptdrr
#with_provinces.head(n=20)
print(with_provinces.shape)

In [ ]:
recent_data_covid19 = pd.concat([with_provinces,us_china_data],join="inner")
print(recent_data_covid19.shape)

In [ ]:

recent_data_covid19.drop("Bahamas, The", axis=0, inplace=True)
recent_data_covid19.drop("The Bahamas",axis=0, inplace=True)
recent_data_covid19.drop(" Azerbaijan",axis=0,inplace=True)
recent_data_covid19.drop("Gambia, The",axis=0, inplace=True)
recent_data_covid19.drop("The Gambia",axis=0, inplace=True)
recent_data_covid19.drop("Republic of the Congo",axis=0,inplace=True)

recent_data_covid19.loc["Bahamas","Confirmed"] = + 86
recent_data_covid19.loc["Azerbaijan","Confirmed"]+=1
recent_data_covid19.loc["Gambia","Confirmed"] += 2

recent_data_covid19.tail(n=10) 


> Pour faire des taux brut j'ai pris un jeux de données avec le nombre d'habitants par pays en 2020


In [ ]:
pop_by_country = pd.read_csv("/kaggle/input/population-by-country-2020/population_by_country_2020.csv")
pop_by_country.rename(columns={"Country (or dependency)":"Country_Region"},inplace=True)
#pop_by_country.set_index(["Country_Region"],drop=False,inplace=True)

pop_by_country.replace("Cape Verde","Cabo Verde",inplace=True)
pop_by_country.replace("Czech Republic (Czechia)","Czech Republic",inplace=True)
pop_by_country.replace("East Timor","Timor-Leste",inplace=True)
pop_by_country.replace("Réunion","Reunion",inplace=True)
pop_by_country.replace("State of Palestine","Palestine",inplace=True)

pop_by_country = pop_by_country.sort_values(by="Country_Region")

pop_by_country.head()
#pop_by_country.shape

> Le problème c'est que dans les deux datasets, un même pays n'a pas forcement le même nom, il peux être écrit en anglais dans un, en français dans l'autre, il y a peut être un espace en plus etc...

In [ ]:
for index,row in recent_data_covid19.iterrows():
    if index not in pop_by_country['Country_Region'].to_list(): # ça va transformer la colonne" name" en liste
        print(index + " n'est pas dans la liste ")
        
        # Ceux que j'ai laissé en plan sont des pays qui ont des ambiguité sur le nom du pays 
        # par rapport au contexte politique

> Les pays que je n'ai pas changés, sont des pays où il y a une ambiguité sur leur statut politique, je ne veux pas fausser les données en ajouter ses pays dans dataset pop_by_country et leur assignant un nombre d'habitant

In [ ]:
recent_data_covid19 = pd.merge(recent_data_covid19,pop_by_country,on= "Country_Region")
dataframe_pop = recent_data_covid19["Population (2020)"].to_frame()
recent_data_covid19.drop(["Yearly Change","Net Change","Density (P/Km²)","Land Area (Km²)","Migrants (net)",\
                          "Fert. Rate","World Share","Med. Age","Urban Pop %","Population (2020)"], axis=1, inplace=True)
print(type(dataframe_pop))

In [ ]:

bold("** EVOLUTION DE LA COVID-19 EN FONCTION DES PAYS**")

# np.round sert à arrondir les nombres
# le deuxieme nombre me donne le nombre de décimal
recent_data_covid19["Taux brut de confirmés (pour 100)"] = np.round(100*recent_data_covid19["Confirmed"]/dataframe_pop["Population (2020)"],4)
recent_data_covid19["Taux de létalité (pour 100)"] = np.round(100*recent_data_covid19["Deaths"]/recent_data_covid19["Confirmed"],2)
recent_data_covid19["Taux de guérisson (pour 100)"] = np.round(100*recent_data_covid19["Recovered"]/recent_data_covid19["Confirmed"],2)

recent_data_covid19.sort_values('Confirmed', ascending= False).style.background_gradient(cmap='Oranges',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Reds',subset=["Taux de létalité (pour 100)"])\
                        .background_gradient(cmap='Greens',subset=["Taux de guérisson (pour 100)"])\
                        .background_gradient(cmap="Oranges",subset=["Taux brut de confirmés (pour 100)"])
                         
# Important: retient ça .background_gradient, subset te permet de donner le nom de la colonne 
# .style donner le genre de discriminant que tu veux

> Il y a quelque chose de frappant, l'allemagne a environ le même nombre de cas que la France cependant elle a 4,5 fois moins de morts que la France, et 3 fois plus de guérris qu'en France.
Une question se pose alors, que font les allemands que nous faisont pas?
Ce seraient intéressant de faire une comparaison 

In [ ]:

# Dataframe pour les 20 pays où la maladie a le plus touché
country_confirmed = pd.DataFrame(recent_data_covid19.loc[:,["Country_Region","Confirmed","Recovered"]])
country_confirmed.sort_values(by='Confirmed',ascending=False, inplace=True)
country_confirmed = country_confirmed.head(n=20)

# Dataframe pour les 15 pays où la maladie a un taux brut de confirmés le plus élévé
country_rate_confirmed = pd.DataFrame(recent_data_covid19.loc[:,['Country_Region','Taux brut de confirmés (pour 100)']])
country_rate_confirmed.sort_values(by='Taux brut de confirmés (pour 100)',ascending=False, inplace=True)
country_rate_confirmed = country_rate_confirmed.head(n=15)

# Dataframe pour les 15 pays où la maladie a le plus haut de mortalité
country_death_rate = pd.DataFrame(recent_data_covid19.loc[:,["Country_Region",'Taux brut de mortalité (pour 100)']])
country_death_rate.sort_values(by="Taux brut de mortalité (pour 100)",ascending=False, inplace=True)
country_death_rate = country_death_rate.head(n=15)

# Dataframe pour les 15 pays avec le plus haut taux de létalité
country_lethality_rate = pd.DataFrame(recent_data_covid19.loc[:,["Country_Region",'Taux de létalité (pour 100)']])
country_lethality_rate.sort_values(by="Taux de létalité (pour 100)",ascending=False, inplace=True)
country_lethality_rate = country_lethality_rate.head(n=15)


In [ ]:
plt.figure(figsize=(12,7))
ax = sns.barplot(x="Confirmed", y="Country_Region", data=country_confirmed)
ax.set(xlabel="Nombre de cas confirmés (10^6)", ylabel="Pays") 
plt.title("TOP 20 DES PAYS LES PLUS TOUCHES PAR LA COVID-19")
plt.show()

> C'est incontesable, il n'y a pas un pays qui est plus touché que les états unis, ce pays ce présente comme un excellent candidat pour la compréhension de la propagation de la maladie.
Dans le top 20 des pays les plus touchés, 9 pays sont Européens, 4 sont d'Amérique, 5 sont des pays asiatiques. Les pays européens sont plus vers le haut de la liste

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2,figsize=(27,10))

# Il faut absolument mettre en copy() sinon python ne sait pas
# si je modifie le dataframe de base, ou si je veux  une copie

country_rate_con1000 = recent_data_covid19.loc[recent_data_covid19["Confirmed"]>1000,:].copy()
country_rate_con1000.sort_values(by="Taux brut de confirmés (pour 100)",ascending=False, inplace=True)
country_rate_con1000 = country_rate_con1000.head(n=15)

country_rate_con5000 = recent_data_covid19.loc[recent_data_covid19["Confirmed"]>5000,:].copy() 
country_rate_con5000.sort_values(by="Taux brut de confirmés (pour 100)",ascending=False, inplace=True)
country_rate_con5000 = country_rate_con5000.head(n=15)

sns.barplot(x='Taux brut de confirmés (pour 100)',y="Country_Region", data=country_rate_con1000, ax=ax1)
ax1.set_title("TAUX BRUT DE CONFIRMES DES PAYS AYANT PLUS DE 1000 CAS DE LA COVID-19")
ax1.set_xlabel("Taux brut de confirmés (pour 100)")

sns.barplot(x="Taux brut de confirmés (pour 100)",y="Country_Region", data=country_rate_con5000, ax=ax2)
ax2.set_title("TAUX BRUT DE CONFIRMES DES PAYS AYANT PLUS DE 5000 CAS DE LA COVID-19")
ax2.set_xlabel("Taux brut de confirmés (pour 100)")

plt.show()

In [ ]:
pd.DataFrame(pop_by_country["Density (P/Km²)"]).describe()

> Le Qatar et le Luxemebourg, arrivent en tête, ce sont tout les deux de petits pays.
Le Qatar a plus de 2 millions d'habitants et une densité de 235 habitants par metres carré.Le Luxembourg Le Qatar et Le Luxembourg se trouve dans le troisieme quartile des pays ayant une plus grande densité par habitant.

In [ ]:
fig, (ax3, ax4) = plt.subplots(1, 2,figsize=(27,10))

#--------------------------------------------------------------------------------------------
country_rate_con10000 = recent_data_covid19[recent_data_covid19["Confirmed"]>10000].copy()
country_rate_con10000.sort_values(by="Taux brut de confirmés (pour 100)",ascending=False, inplace=True)
country_rate_con10000 = country_rate_con10000.head(n=15)

country_rate_con100000 = recent_data_covid19[recent_data_covid19["Confirmed"]>100000].copy()
country_rate_con100000.sort_values(by="Taux brut de confirmés (pour 100)",ascending=False, inplace=True)
country_rate_con100000 = country_rate_con100000.head(n=15)

sns.barplot(x='Taux brut de confirmés (pour 100)',y="Country_Region", data=country_rate_con10000, ax=ax3)
ax3.set_title("TAUX BRUT DE CONFIRMES PAR PAYS AYANT PLUS DE 10000 CAS (POUR 100) ")
ax3.set_xlabel("Taux brut de confirmés (pour 1000)")

sns.barplot(x="Taux brut de confirmés (pour 100)",y="Country_Region",data=country_rate_con100000, ax=ax4)
ax4.set_title("TAUX BRUT DE CONFIRMES PAR PAYS AYANT PLUS DE 100000 CAS (POUR 100) ")
ax4.set_xlabel("Taux brut de confirmés (pour 100)")

plt.show()

> Le problème avec le taux brut de confirmés, c'est qu'il ne traduit pas la situation réelle, la COVID-19 ne touche pas un pays de manière uniforme, il y a certain endroits, à l'échelle meme d'un pays où la COVID-19 touche plus certain régions que d'autres, nous ne prendrons donc pas cette variable en compte

In [ ]:
fig, (ax5, ax6) = plt.subplots(1, 2,figsize=(27,10))


#----------------------------------------------------------------------------------------------
country_rate_reco1000 = recent_data_covid19[recent_data_covid19["Confirmed"]>1000].copy()
country_rate_reco1000.sort_values(by="Taux de létalité (pour 100)",ascending=False, inplace=True)
country_rate_reco1000 = country_rate_reco1000.head(n=15)

country_rate_reco5000 = recent_data_covid19[recent_data_covid19["Confirmed"]>5000].copy()
country_rate_reco5000.sort_values(by="Taux de létalité (pour 100)",ascending=False, inplace=True)
country_rate_reco5000 = country_rate_reco5000.head(n=15)

sns.barplot(x='Taux de létalité (pour 100)',y="Country_Region", data=country_rate_reco1000,ax=ax5)
ax5.set_title("TAUX DE LETALITE POUR LES PAYS AYANT PLUS DE 1000 CAS (POUR 100)")
ax5.set_xlabel("Taux de létalité (pour 100)")

sns.barplot(x="Taux de létalité (pour 100)",y="Country_Region",data=country_rate_reco5000, ax=ax6)
ax6.set_title("TAUX DE LETALITE POUR LES PAYS AYANT PLUS DE 5000 CAS (POUR 100)")
ax6.set_xlabel("Taux de létalité (pour 100)")

plt.show()

> La Belgique se place en premier position, suivi du Royaume-Uni, de la France,  et de  de l'Italie.
On va sélectionner la France, comme nous sommes français et que la France est le 5eme pays ayant le plus de cas.


In [ ]:
fig, (ax7,ax8) = plt.subplots(1, 2,figsize=(27,10))

#----------------------------------------------------------------------------------------
country_reco1000 = recent_data_covid19[recent_data_covid19["Confirmed"]>1000].copy()
country_reco1000.sort_values(by="Taux de guérisson (pour 100)",ascending=False, inplace=True)
country_reco1000 = country_reco1000.head(n=15)

country_reco5000 = recent_data_covid19[recent_data_covid19["Confirmed"]>5000].copy()
country_reco5000.sort_values(by="Taux de guérisson (pour 100)",ascending=False, inplace=True)
country_reco5000 = country_reco5000.head(n=15)

sns.barplot(x='Taux de guérisson (pour 100)',y="Country_Region", data=country_reco1000,ax=ax7)
ax7.set_title("TAUX DE GUERISSON POUR LES PAYS AYANT PLUS DE 1000 CAS (POUR 100)")
ax7.set_xlabel("Taux de mortalié (pour 100)")

sns.barplot(x="Taux de guérisson (pour 100)",y="Country_Region",data=country_reco5000, ax=ax8)
ax8.set_title("TAUX DE GUERISSON POUR LES PAYS AYANT PLUS DE 5000 CAS (POUR 100)")
ax8.set_xlabel("Taux de guérisson (pour 100)")

plt.show()

> Etonnament,c'est en Chine et en Korée du Sud, que les taux de guérisson sont les plus élévé. On rappelle que c'est la Chine qui a été la premier touché par le virus. Alors comment on t-ils fait?

> 

> #     **France, Italie, Etats-Unis versus Chine, Allemagne, Corée du sud.     Qu'est-ce qui différencie les bons élèves des mauvais?**

> Dans notre précédente analyse, on a vu d'une part que la France et l'Italie se classent comme étant des pays ayant les taux de létalité les plus haut. Les Etats-Unis est de loin le pays le plus touché par le virus.
D'autre, on remarque que malgré le fait que la Chine est été surprise par le virus, elle est l'un des pays qui se débrouille le mieux.
La Corée du Sud, compte environ 255 décès pour une population de 52 millions d'habitants.
L'allemagne se débrouille plutôt bien lorsqu'on le compare à ses voisins. On rappelle que l'Allemagne a peu près le même nombre de cas que la France. Cependant,l'Allemagne a trois fois plus de guérris, et 4-5 fois moins de morts.


In [ ]:
global_confirmed = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
global_confirmed.head(60)

> Ce dataset contient une colonne avec les pays,les provinces, une colonne pour la latitude et une colonne avec la longitude, et les autres colonnes sont les données cumulés au niveau des provinces.
Donc comme la dernière fois, lorsque le pays contient une province ces confirmés sont comptés par provinces

In [ ]:
# Je veux regrouper les données par provinces
global_confirmed = global_confirmed.drop(columns = ["Lat","Long"])
global_confirmed.rename(columns={"Country/Region":"Country_Region"},inplace=True)
global_confirmed.rename(columns={"Province/State":"Province_State"},inplace=True)

#Je garde ça on sait jamais je peux en avoir besoin
global_confirmed_with_provinces = global_confirmed[pd.isnull(global_confirmed["Province_State"])== False]

global_confirmed_with_provinces.head()

In [ ]:
# Je veux regrouper les données par pays 
global_confirmed = global_confirmed.groupby('Country_Region').sum()
global_confirmed.head(10)

In [ ]:
#Je prend la transposée pour pouvoir ploter
global_confirmed_transposed = global_confirmed.T
global_confirmed_transposed.head()

In [ ]:

global_confirmed_transposed.plot(y=['Korea, South','Italy','China','US','France','Germany'],\
                                 figsize=(12,7),use_index=True, marker="*")
plt.xlabel("Dates")
plt.ylabel("Nombre de cas confirmés cumulés (10^6)")
plt.title("Evolution des cas confirmés au cours du temps, mois/jour/année")
plt.show()


> On voit que les chiffres de nos deux datasets coïncident c'est agréable

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.sort_values(by='name',inplace=True)
#world.plot(figsize=(12,6))
#plt.show()

> Encore une fois, je dois faire en sorte que les pays aient le même nom dans les deux datasets

In [ ]:
for index,row in global_confirmed.iterrows():
    if index not in world['name'].to_list(): # ça va transformer la colonne" name" en liste
        print(index + " n'est pas dans la liste ")

> Maintenant, je change les noms dans les datasets

In [ ]:

# Il n'y a pas Andorra dans world
# Il n'y a pas Antigua et Barbuda

world.replace("Bosnia and Herz.","Bosnia and Herzegovina",inplace =True)

# Il n'y a pas Burma dans world
# Il n'y a pas Cabo Verde

world.replace("Central African Rep.","Central African Republic", inplace=True)
global_confirmed.replace("Congo (Brazzaville)","Congo", inplace=True)
global_confirmed.replace("Congo (Kinshasa)","Dem. Rep. Congo",inplace=True)
global_confirmed.replace("Cote d'Ivoire","Côte d'Ivoire	", inplace=True)

# Diamond Princess n'est même pas un pays
# Dominica n'est pas dans world

world.replace("Dominican Republic","Dominican Rep.",inplace=True)
world.replace("Eq. Guinea","Equatorial Guinea",inplace=True)
world.replace("eSwatini","Eswatini",inplace=True)

# Il n'y a pas Grenada dans world
# Holy see c'est pas dans la carte, mais c'est en italie

global_confirmed.replace("Korea, South","South Korea", inplace=True)

# Le Liechtenstein n'est pas dans World
# MS Zaandam est un navire de croisiere
# Maldives n'est pas dans world
# Malta n'est pas dans world
# Mauritus n'est pas dans world
# Monaco n'est pas dans world

global_confirmed.replace("North Macedonia","Macedonia",inplace= True)

# St Kitts pas dans world
# St Lucia """"""""""
# St Vincent and the Grenadines
# San Marino pas ds world
# Seychelles
# Singapore 

world.replace("S. Sudan","South Sudan",inplace=True)
global_confirmed.replace("Taiwan*","Taiwan",inplace=True)
world.replace("United States of America","US",inplace = True)

# West Bank and Gaza je vais le placer en Israël

world.replace("W.Sahara","Western Sahara",inplace=True)


# On va laisser ça comme ça, les petits pays sont à peine visible sur la carte, ça va rien changer 
# à la compréhension de l'expansion de la maladie



> Commençons par la chine, c'est la que tout a commencé